# Workflow for the Backmapping method

![title](./bcml.png)

# Table of Contents

* <a href='#a'> Import the needed libraries for the task </a>
* <a href='#pre-processing'> Pre-processing 
    - <a href='#c'> Load the chemical structure of the system </a>
    - <a href='#d'> Load the trajectory file </a>
    - <a href='#e'> Compute the number of particles per chain  </a>
    - <a href='#g'> Compute the bond vectors for each monomer type  </a>
    - <a href='#h'> Define the function that generates the target and input of the CNN  </a>
    - <a href='#f'> Create CNN input and target output for a test-set configuration  </a>
* <a href='#training-process'> Backmapping by utilizing the trained CNN    
   * <a href='#j'> Load the data </a>
   * <a href='#i'> Develop the conditional convolutional neural network  </a>
   * <a href='#l'> Decoding process  </a>
    - <a href='#m'> Decoding the output of the neural network for each CG type </a>
    - <a href='#n'> Get the structure of each chain </a>
    - <a href='#o'> Re-insert atomic detail to CG congigurations via the trained CNN </a>
   * <a href='#p'> Prediction of an atomistic configuration from a given CG configuration  </a> 
* <a href='#q'> Visualize the results </a>
    - <a href='#r'> Generate the distribution plots for bond lengths, bond angles and dihedral angles </a>

# Import the needed libraries for this task  <span id='a'/>

In [59]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import os
import mdtraj as md
from copo import load_chemistry
from params import *
import pickle
import time
import random
from matplotlib.ticker import FormatStrFormatter
from collections import defaultdict
import seaborn as sns
import warnings
from distribution_plots import generate_distribution_plots
# turn off the warnings 
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

# Pre-processing <span id='pre-processing'/>

### Load the chemical structure of the system <span id='c'/>

In [45]:
chemistry_filename = '../Data/chem_tcv_451045.txt'
chemistry, chemistry_names = load_chemistry(chemistry_filename, None)
chemistry = np.array(chemistry)
print("Number of Chains per configuration:",chemistry.shape[0])
print("Number of Monomers per chain:",chemistry.shape[1])

Read 96 molecules
Number of Chains per configuration: 96
Number of Monomers per chain: 30


### Load the trajectory file <span id='d'/>

In [46]:
t = md.load('../Data/trajout_long.gro')
print(t)

<mdtraj.Trajectory with 1851 frames, 11562 atoms, 96 residues, and unitcells>


###  Compute the number of particles per chain  <span id='e'/>

In [47]:
def calc_chainlens(chemistry,merlen_cis,merlen_trans,merlen_vinyl):
    chainlens = []
    for i in range(chemistry.shape[0]):
      ntypes = [np.count_nonzero(chemistry[i] == 0),np.count_nonzero(chemistry[i] == 1),np.count_nonzero(chemistry[i] == 2)] 
      chainlen = ntypes[0]*merlen_cis + ntypes[1]*merlen_trans + ntypes[2]*merlen_vinyl
      chainlens.append(int(chainlen))
    
    chainlens = np.array(chainlens, dtype=np.uint8)
    np.savetxt("./chainlens.txt",chainlens)
    return chainlens  

chainlens = calc_chainlens(chemistry,merlen_cis,merlen_trans,merlen_vinyl)

### Compute the bond vectors for each monomer type  <span id='g'/>

In [48]:
def cis_vectors(index,imageAT,Coord,jj,b0):
    bv1=Coord[1+index]-Coord[index]
    bv2=Coord[2+index]-Coord[1+index]
    bv3=Coord[3+index]-Coord[2+index]
    if jj != nmer-1:
      bv0=Coord[4+index]-Coord[3+index]
    else:
      bv0= b0[:]
    imageAT.append([bv1[0],bv1[1],bv1[2]])
    imageAT.append([bv2[0],bv2[1],bv2[2]])
    imageAT.append([bv3[0],bv3[1],bv3[2]])      
    imageAT.append([bv0[0],bv0[1],bv0[2]])
    return bv0     

def trans_vectors(index,imageAT,Coord,jj,b0):
    bv1=Coord[1+index]-Coord[index]
    bv2=Coord[2+index]-Coord[1+index]
    bv3=Coord[3+index]-Coord[2+index]
    if jj != nmer-1:
      bv0=Coord[4+index]-Coord[3+index]
    else:
      bv0= b0[:]
    imageAT.append([bv1[0],bv1[1],bv1[2]])
    imageAT.append([bv2[0],bv2[1],bv2[2]])
    imageAT.append([bv3[0],bv3[1],bv3[2]])      
    imageAT.append([bv0[0],bv0[1],bv0[2]])
    return bv0     

def vinyl_vectors(index,imageAT,Coord,jj,b0):
    bv1=Coord[1+index]-Coord[index]
    bv2=Coord[2+index]-Coord[1+index]
    bv3=Coord[3+index]-Coord[2+index]
    if jj != nmer-1:
      bv0=Coord[4+index]-Coord[1+index]
    else:
      bv0= b0[:]    
    imageAT.append([bv1[0],bv1[1],bv1[2]])
    imageAT.append([bv2[0],bv2[1],bv2[2]])
    imageAT.append([bv3[0],bv3[1],bv3[2]])      
    imageAT.append([bv0[0],bv0[1],bv0[2]])
    return bv0     

### Define the function that generates the target and input of the CNN <span id='h'/>

In [49]:
def encoding(frames,save_path,t,chemistry):  
 input_file=[]
 target_file=[]  
 b0 = np.zeros((3))
 for frameIndx in frames:
    # Counter to ignore the last particle of the chain, when the chain ends in vinly-1,2
    counter=0
    
    # Compute the length of the box
    LXX,LYY,LZZ=t.unitcell_lengths[frameIndx][0],t.unitcell_lengths[frameIndx][1],t.unitcell_lengths[frameIndx][2]
    
    hLXX,hLYY,hLZZ=LXX/2.0,LYY/2.0,LZZ/2.0
    Coord=np.zeros([npart,3],dtype=np.float32)
    
    # Counter for the particles of the frame
    frame_counter_1 = -1
    
    frame_counter_2 = 0
    for j in range(nchain):
        coordCG=np.zeros([nmer,3],dtype=np.float32)
        coordAT = []  
        for jj in range(nmer):
            # Compute the type of the monomer
            type = chemistry[j][jj]
            
            posCM=[0.,0.,0.]
            if(type==0): masses,merlen = masses_cis,merlen_cis
            elif(type==1): masses,merlen = masses_trans,merlen_trans
            elif(type==2): masses,merlen = masses_vinyl,merlen_vinyl
            totmass=np.sum(masses)
            for ii in range(merlen):
                frame_counter_1 += 1
                partIndx = frame_counter_1
                
                # Compute the un-wrapped coordinates of the particles
                Coord[partIndx]=t.xyz[frameIndx,(partIndx+counter),:]
                if (jj!=0 or ii!=0):
                    if(Coord[partIndx][0]-Coord[partIndx-1][0]<-hLXX): Coord[partIndx][0]+=LXX
                    if(Coord[partIndx][0]-Coord[partIndx-1][0]>hLXX): Coord[partIndx][0]-=LXX
                    if(Coord[partIndx][1]-Coord[partIndx-1][1]<-hLYY): Coord[partIndx][1]+=LYY
                    if(Coord[partIndx][1]-Coord[partIndx-1][1]>hLYY): Coord[partIndx][1]-=LYY
                    if(Coord[partIndx][2]-Coord[partIndx-1][2]<-hLZZ): Coord[partIndx][2]+=LZZ
                    if(Coord[partIndx][2]-Coord[partIndx-1][2]>hLZZ): Coord[partIndx][2]-=LZZ
           
            # Compute the coordinates of the CG particles of the chain 
                posCM[0]+=Coord[partIndx][0]*masses[ii]
                posCM[1]+=Coord[partIndx][1]*masses[ii]
                posCM[2]+=Coord[partIndx][2]*masses[ii]

            posCM[0]/=totmass
            posCM[1]/=totmass
            posCM[2]/=totmass
            # Save the coordinates of the CG particles of the chain 
            coordCG[jj]=posCM[:]
            
        # Ignore the last particle of the chain, when the chain ends in vinyl-1,2    
        if(j in cv3): counter+=1
        
        for jj in range(nmer):
            type = chemistry[j][jj]
            
            # Compute the bond vectors for each monomer 
            if(type==0):
                merlen = merlen_cis
                b0[:]=cis_vectors(frame_counter_2,coordAT,Coord,jj,b0)
            elif(type==1): 
                merlen = merlen_trans
                b0[:]=trans_vectors(frame_counter_2,coordAT,Coord,jj,b0)
            elif(type==2):
                merlen = merlen_vinyl  
                b0[:]=vinyl_vectors(frame_counter_2,coordAT,Coord,jj,b0)

            frame_counter_2 += merlen

        coordAT = np.array(coordAT,dtype=np.float64)                
        
        # Compute the number of splits (samples) per chain 
        nsplits = int(np.ceil((coordAT.shape[0])/INPUT_WIDTH))  
        s = int(np.ceil(nmer/nsplits)) 
        if (s*(4)) > INPUT_WIDTH: nsplits += 1
        s = int(np.ceil(nmer/nsplits))
        meridx = [(s*(h+1)-1) for h in range(nsplits)]
        meridx.pop(-1)
        meridx.append(nmer-1)
        AT = []
        CG = []
        frame_counter = -1
        arrAT = np.zeros([INPUT_WIDTH,3],dtype=np.float64)
        arrCG = np.zeros([INPUT_WIDTH,6],dtype=np.float64)
        for jj in range(nmer):
            type = chemistry[j][jj]
            if(type==0): 
                dtseg = dtseg_cis
                m_id = [0,0,1]
            elif(type==1):
                dtseg = dtseg_trans
                m_id = [0,1,0]
            elif(type==2): 
                dtseg = dtseg_vinyl
                m_id = [1,0,0]
            for g in range(dtseg):
                frame_counter += 1
        
                AT.append([coordAT[frame_counter][0],coordAT[frame_counter][1],coordAT[frame_counter][2]])
                CG.append([coordCG[jj][0],coordCG[jj][1],coordCG[jj][2],m_id[0],m_id[1],m_id[2]])
            
            # Create input and target samples 
            if jj in meridx: 
                AT = np.array(AT,dtype=np.float64)
                CG = np.array(CG,dtype=np.float64)
                
                # Compute the zero-padding 
                shiftx = int(np.ceil((INPUT_WIDTH - AT.shape[0])/2))
                
                arrAT[shiftx:shiftx+int(AT.shape[0])] = AT[:]
                arrCG[shiftx:shiftx+int(CG.shape[0])] = CG[:]  
                input_file.append(arrCG)
                target_file.append(arrAT)                
                AT = []
                CG = []
                arrAT = np.zeros([INPUT_WIDTH,3],dtype=np.float64)
                arrCG = np.zeros([INPUT_WIDTH,6],dtype=np.float64)
                
 final_input=np.array(input_file,dtype=np.float64)
 final_target=np.array(target_file,dtype=np.float64) 
 print("Input shape:", final_input.shape)
 print("Target output shape:", final_target.shape)      
 
 with open(save_path+'_input.pkl','wb') as f:
     pickle.dump(final_input, f)

 with open(save_path+'_target.pkl','wb') as f:
     pickle.dump(final_target, f)        
 input_file = []
 target_file = []


### Create CNN inputs and target outputs for a test-set configuration <span id='f'/>

In [50]:
random.seed(100)
frames = random.sample(range(1851), 1851)

# Configurations of the test-set
test_frames = frames[1666:]

# Choose a configuration of the test-set
save_path="test" 
encoding([test_frames[0]],save_path,t,chemistry) 

test_frame = test_frames[0]

print("Create CNN input and target output for configuration:",test_frame)

Input shape: (96, 128, 6)
Target output shape: (96, 128, 3)
Create CNN input and target output for configuration: 525


# Backmapping by utilizing the trained CNN   <span id='training-process'/>

### Load the data <span id='j'/>

In [51]:
with open('./test_target.pkl','rb') as f:
    test_target = pickle.load(f)
    print("Shape of target output for a single configuration:",test_target.shape)    

with open('./test_input.pkl','rb') as f:
    test_input = pickle.load(f)
    print("Shape of input for a single configuration:",test_input.shape)        

print("Number of samples:",test_input.shape[0])    

Shape of target output for a single configuration: (96, 128, 3)
Shape of input for a single configuration: (96, 128, 6)
Number of samples: 96


### Develop the conditional convolutional neural network  <span id='i'/>

In [52]:
OUTPUT_CHANNELS = 3

# Downsample blog
def downsample(entered_input,filters, size, apply_batchnorm=True,strides=2):
  
  conv1 = tf.keras.layers.Conv1D(filters, size, strides=strides, padding='same',use_bias=False)(entered_input) 
  conv1 = tf.keras.layers.LeakyReLU()(conv1)
  
  if apply_batchnorm:
    conv1 = tf.keras.layers.BatchNormalization()(conv1)

  return conv1

# Upsample blog
def upsample(entered_input,filters, size, skip_layer, apply_dropout=False, strides=2, apply_skip=True):
  tran1 = tf.keras.layers.Conv1DTranspose(filters, size, strides=strides, padding='same', use_bias=True)(entered_input)
  tran1 = tf.keras.layers.ReLU()(tran1) 
  if apply_dropout:
      tran1 = tf.keras.layers.Dropout(0.5)(tran1)
  
  if apply_skip:
      tran1 = tf.keras.layers.Concatenate()([tran1,skip_layer])
  return tran1

# Create the Convolutional Neural Network (CNN)
def Generator(input_size): 
  input1 = tf.keras.layers.Input(input_size)  
  output1 = downsample(input1, 64, 3)
  output2 = downsample(output1, 128, 3)
  output3 = downsample(output2, 256, 3)  
  output4 = downsample(output3, 512, 3) 
  output5 = downsample(output4, 512, 3) 
  
  output = upsample(output5, 512, 3, output4, apply_dropout=True)
  output = upsample(output, 256, 3, output3, apply_dropout=False)
  output = upsample(output, 128, 3, output2, apply_dropout=False)
  output = upsample(output, 64, 3, output1, apply_dropout=False)
  
  output = tf.keras.layers.Conv1DTranspose(64, 3, strides=2, padding="same",  activation="relu")(output)
  out = tf.keras.layers.Conv1DTranspose(3, 3, strides=1, padding="same",  activation="tanh")(output)

  model = tf.keras.models.Model(input1,out)
  return model

In [53]:
input_size = (INPUT_WIDTH,6)

model = Generator(input_size)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128, 6)]     0                                            
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, 64, 64)       1152        input_4[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_15 (LeakyReLU)      (None, 64, 64)       0           conv1d_15[0][0]                  
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 64, 64)       256         leaky_re_lu_15[0][0]             
____________________________________________________________________________________________

# Decoding process <span id='l'/>

### Decoding the output of the neural network for each CG type <span id='m'/>

In [54]:
def cis_d(pos,masses,iosx,iosy,iosz,b0,nmer_count,Coords,identities,PartIndx):          
     b1 = [pos[0,0],pos[0,1],pos[0,2]]
     b2 = [pos[1,0],pos[1,1],pos[1,2]]
     b3 = [pos[2,0],pos[2,1],pos[2,2]]
     b0[nmer_count][:] = [pos[3,0],pos[3,1],pos[3,2]]
     vcoords = np.zeros((5,3))
     totmass = np.sum(masses)
     if nmer_count == 0:  
       posx=-(masses[1]*b1[0]+masses[2]*(b1[0]+b2[0])+masses[3]*(b1[0]+b2[0]+b3[0]))/totmass
       posy=-(masses[1]*b1[1]+masses[2]*(b1[1]+b2[1])+masses[3]*(b1[1]+b2[1]+b3[1]))/totmass
       posz=-(masses[1]*b1[2]+masses[2]*(b1[2]+b2[2])+masses[3]*(b1[2]+b2[2]+b3[2]))/totmass
      
       vcoords[1][:] = [posx,posy,posz]
       vcoords[2][:] = [vcoords[1][0]+b1[0],vcoords[1][1]+b1[1],vcoords[1][2]+b1[2]]     
       vcoords[3][:] = [vcoords[2][0]+b2[0],vcoords[2][1]+b2[1],vcoords[2][2]+b2[2]]    
       vcoords[4][:] = [vcoords[3][0]+b3[0],vcoords[3][1]+b3[1],vcoords[3][2]+b3[2]]      
     
     elif nmer_count != 0: 
       posx=-(b0[nmer_count-1][0]*masses[0]+masses[1]*(b0[nmer_count-1][0]+b1[0])+masses[2]*(b0[nmer_count-1][0]+b1[0]+b2[0])+masses[3]*(b0[nmer_count-1][0]+b1[0]+b2[0]+b3[0]))/totmass
       posy=-(b0[nmer_count-1][1]*masses[0]+masses[1]*(b0[nmer_count-1][1]+b1[1])+masses[2]*(b0[nmer_count-1][1]+b1[1]+b2[1])+masses[3]*(b0[nmer_count-1][1]+b1[1]+b2[1]+b3[1]))/totmass
       posz=-(b0[nmer_count-1][2]*masses[0]+masses[1]*(b0[nmer_count-1][2]+b1[2])+masses[2]*(b0[nmer_count-1][2]+b1[2]+b2[2])+masses[3]*(b0[nmer_count-1][2]+b1[2]+b2[2]+b3[2]))/totmass  
       
       vcoords[0][:] = [posx,posy,posz]
       vcoords[1][:] = [vcoords[0][0]+b0[nmer_count-1][0],vcoords[0][1]+b0[nmer_count-1][1],vcoords[0][2]+b0[nmer_count-1][2]]   
       vcoords[2][:] = [vcoords[1][0]+b1[0],vcoords[1][1]+b1[1],vcoords[1][2]+b1[2]]  
       vcoords[3][:] = [vcoords[2][0]+b2[0],vcoords[2][1]+b2[1],vcoords[2][2]+b2[2]]  
       vcoords[4][:] = [vcoords[3][0]+b3[0],vcoords[3][1]+b3[1],vcoords[3][2]+b3[2]]  
         
     Coords[PartIndx][:] = [iosx+vcoords[1][0],iosy+vcoords[1][1],iosz+vcoords[1][2]]
     Coords[PartIndx+1][:] = [iosx+vcoords[2][0],iosy+vcoords[2][1],iosz+vcoords[2][2]]
     Coords[PartIndx+2][:] = [iosx+vcoords[3][0],iosy+vcoords[3][1],iosz+vcoords[3][2]]
     Coords[PartIndx+3][:] = [iosx+vcoords[4][0],iosy+vcoords[4][1],iosz+vcoords[4][2]]     

     return 

def trans_d(pos,masses,iosx,iosy,iosz,b0,nmer_count,Coords,identities,PartIndx):  
     b1 = [pos[0,0],pos[0,1],pos[0,2]]
     b2 = [pos[1,0],pos[1,1],pos[1,2]]
     b3 = [pos[2,0],pos[2,1],pos[2,2]]
     b0[nmer_count][:] = [pos[3,0],pos[3,1],pos[3,2]]
     vcoords = np.zeros((5,3))
     totmass = np.sum(masses)
     if nmer_count == 0:  
       posx=-(masses[1]*b1[0]+masses[2]*(b1[0]+b2[0])+masses[3]*(b1[0]+b2[0]+b3[0]))/totmass
       posy=-(masses[1]*b1[1]+masses[2]*(b1[1]+b2[1])+masses[3]*(b1[1]+b2[1]+b3[1]))/totmass
       posz=-(masses[1]*b1[2]+masses[2]*(b1[2]+b2[2])+masses[3]*(b1[2]+b2[2]+b3[2]))/totmass
      
       vcoords[1][:] = [posx,posy,posz]
       vcoords[2][:] = [vcoords[1][0]+b1[0],vcoords[1][1]+b1[1],vcoords[1][2]+b1[2]]     
       vcoords[3][:] = [vcoords[2][0]+b2[0],vcoords[2][1]+b2[1],vcoords[2][2]+b2[2]]    
       vcoords[4][:] = [vcoords[3][0]+b3[0],vcoords[3][1]+b3[1],vcoords[3][2]+b3[2]]
     
     elif nmer_count != 0: 
       posx=-(b0[nmer_count-1][0]*masses[0]+masses[1]*(b0[nmer_count-1][0]+b1[0])+masses[2]*(b0[nmer_count-1][0]+b1[0]+b2[0])+masses[3]*(b0[nmer_count-1][0]+b1[0]+b2[0]+b3[0]))/totmass
       posy=-(b0[nmer_count-1][1]*masses[0]+masses[1]*(b0[nmer_count-1][1]+b1[1])+masses[2]*(b0[nmer_count-1][1]+b1[1]+b2[1])+masses[3]*(b0[nmer_count-1][1]+b1[1]+b2[1]+b3[1]))/totmass
       posz=-(b0[nmer_count-1][2]*masses[0]+masses[1]*(b0[nmer_count-1][2]+b1[2])+masses[2]*(b0[nmer_count-1][2]+b1[2]+b2[2])+masses[3]*(b0[nmer_count-1][2]+b1[2]+b2[2]+b3[2]))/totmass  
       
       vcoords[0][:] = [posx,posy,posz]
       vcoords[1][:] = [vcoords[0][0]+b0[nmer_count-1][0],vcoords[0][1]+b0[nmer_count-1][1],vcoords[0][2]+b0[nmer_count-1][2]]   
       vcoords[2][:] = [vcoords[1][0]+b1[0],vcoords[1][1]+b1[1],vcoords[1][2]+b1[2]]  
       vcoords[3][:] = [vcoords[2][0]+b2[0],vcoords[2][1]+b2[1],vcoords[2][2]+b2[2]]  
       vcoords[4][:] = [vcoords[3][0]+b3[0],vcoords[3][1]+b3[1],vcoords[3][2]+b3[2]]  
         
     Coords[PartIndx][:] = [iosx+vcoords[1][0],iosy+vcoords[1][1],iosz+vcoords[1][2]]
     Coords[PartIndx+1][:] = [iosx+vcoords[2][0],iosy+vcoords[2][1],iosz+vcoords[2][2]]
     Coords[PartIndx+2][:] = [iosx+vcoords[3][0],iosy+vcoords[3][1],iosz+vcoords[3][2]]
     Coords[PartIndx+3][:] = [iosx+vcoords[4][0],iosy+vcoords[4][1],iosz+vcoords[4][2]]     

     return 
     
def vinyl_d(pos,masses,iosx,iosy,iosz,b0,nmer_count,Coords,identities,PartIndx):  
     b1 = [pos[0,0],pos[0,1],pos[0,2]]
     b2 = [pos[1,0],pos[1,1],pos[1,2]]
     b3 = [pos[2,0],pos[2,1],pos[2,2]]
     b0[nmer_count][:] = [pos[3,0],pos[3,1],pos[3,2]]
     vcoords = np.zeros((5,3))
     totmass = np.sum(masses)
     if nmer_count == 0:  
       posx=-(masses[1]*b1[0]+masses[2]*(b1[0]+b2[0])+masses[3]*(b1[0]+b2[0]+b3[0]))/totmass
       posy=-(masses[1]*b1[1]+masses[2]*(b1[1]+b2[1])+masses[3]*(b1[1]+b2[1]+b3[1]))/totmass
       posz=-(masses[1]*b1[2]+masses[2]*(b1[2]+b2[2])+masses[3]*(b1[2]+b2[2]+b3[2]))/totmass
      
       vcoords[1][:] = [posx,posy,posz]
       vcoords[2][:] = [vcoords[1][0]+b1[0],vcoords[1][1]+b1[1],vcoords[1][2]+b1[2]]     
       vcoords[3][:] = [vcoords[2][0]+b2[0],vcoords[2][1]+b2[1],vcoords[2][2]+b2[2]]    
       vcoords[4][:] = [vcoords[3][0]+b3[0],vcoords[3][1]+b3[1],vcoords[3][2]+b3[2]]
       
     elif nmer_count != 0: 
       posx=-(b0[nmer_count-1][0]*masses[0]+masses[1]*(b0[nmer_count-1][0]+b1[0])+masses[2]*(b0[nmer_count-1][0]+b1[0]+b2[0])+masses[3]*(b0[nmer_count-1][0]+b1[0]+b2[0]+b3[0]))/totmass
       posy=-(b0[nmer_count-1][1]*masses[0]+masses[1]*(b0[nmer_count-1][1]+b1[1])+masses[2]*(b0[nmer_count-1][1]+b1[1]+b2[1])+masses[3]*(b0[nmer_count-1][1]+b1[1]+b2[1]+b3[1]))/totmass
       posz=-(b0[nmer_count-1][2]*masses[0]+masses[1]*(b0[nmer_count-1][2]+b1[2])+masses[2]*(b0[nmer_count-1][2]+b1[2]+b2[2])+masses[3]*(b0[nmer_count-1][2]+b1[2]+b2[2]+b3[2]))/totmass  
       
       vcoords[0][:] = [posx,posy,posz]
       vcoords[1][:] = [vcoords[0][0]+b0[nmer_count-1][0],vcoords[0][1]+b0[nmer_count-1][1],vcoords[0][2]+b0[nmer_count-1][2]]   
       vcoords[2][:] = [vcoords[1][0]+b1[0],vcoords[1][1]+b1[1],vcoords[1][2]+b1[2]]  
       vcoords[3][:] = [vcoords[2][0]+b2[0],vcoords[2][1]+b2[1],vcoords[2][2]+b2[2]]  
       vcoords[4][:] = [vcoords[3][0]+b3[0],vcoords[3][1]+b3[1],vcoords[3][2]+b3[2]]  
         
     Coords[PartIndx][:] = [iosx+vcoords[1][0],iosy+vcoords[1][1],iosz+vcoords[1][2]]
     Coords[PartIndx+1][:] = [iosx+vcoords[2][0],iosy+vcoords[2][1],iosz+vcoords[2][2]]
     Coords[PartIndx+2][:] = [iosx+vcoords[3][0],iosy+vcoords[3][1],iosz+vcoords[3][2]]
     Coords[PartIndx+3][:] = [iosx+vcoords[4][0],iosy+vcoords[4][1],iosz+vcoords[4][2]]     

     return 

### Get the structure of each chain <span id='n'/>

In [55]:
def mer_identity(inp):  
  # Compute the zero-padding  
  shiftx = [i for i, rgb in enumerate(inp) if rgb[0] != 0 and rgb[1] != 0 and rgb[2] != 0][0]   
  identities = []
  names = []
  cis_names = ["CC","CDC","CDC","CC"]
  trans_names = ["CT","CDT","CDT","CT"]
  vinyl_names = ["CV2","CV1","CDV1","CDV2"]   
  f = 0
  # Find the sequence of CG types
  for i in range(inp.shape[0]):
      if (inp[i,5]==1 and (abs(inp[i,0])+abs(inp[i,1])+abs(inp[i,2])) != 0) :
          f += 1
          if f%dtseg_cis != 0: 
              continue
          else:    
              identities.append(int(0))
              names += cis_names
      elif (inp[i,4]==1 and (abs(inp[i,0])+abs(inp[i,1])+abs(inp[i,2])) != 0):  
          f += 1 
          if f%dtseg_trans != 0: 
              continue
          else:    
              identities.append(int(1))
              names += trans_names
      elif (inp[i,3]==1 and (abs(inp[i,0])+abs(inp[i,1])+abs(inp[i,2])) != 0):  
          f += 1 
          if f%dtseg_vinyl != 0: 
              continue
          else:
              identities.append(int(2))
              names += vinyl_names
  # Compute the number of monomers in this sample            
  img_nmer = len(identities)      
  return identities, shiftx, img_nmer, names

### Re-insert atomic detail to CG congigurations via the trained CNN <span id='o'/>

In [56]:
def ua_idx(shiftx,c,j):
    return shiftx + c + j

def create_chains(test_frame):
 # Length of the simulation box   
 LXX,LYY,LZZ= 6.73965 , 6.73965 , 6.73965 
 hLXX,hLYY,hLZZ=LXX/2.0,LYY/2.0,LZZ/2.0
 
 # Counter for the test-frames index
 nframes = 0
 
 # Number of particles per chain
 chainlens = np.loadtxt('./chainlens.txt')
 save_path="./Data/"
 os.mkdir(save_path)
 pdimg=0
 chemistry = []
 chain_names = []
 nmer_count = 0
 PartIndx = 0
 pb0=np.zeros([nmer,3],dtype=np.float64)  
 tb0=np.zeros([nmer,3],dtype=np.float64)  
 for inp, tar in zip(test_input,test_target):
  inp = np.reshape(inp,(1,INPUT_WIDTH,6))   
  if len(chemistry) == 0:   
    Pcoords = np.zeros([int(chainlens[pdimg]),3],dtype=np.float64)
    Tcoords = np.zeros([int(chainlens[pdimg]),3],dtype=np.float64)
    if pdimg == 0:
     Pst = open(save_path+'PStart_'+str(test_frame)+'.gro', 'w')
     Pst.write("PB_TCV_451045\n")
     Pst.write(str(npart)+"\n")
    
     pstd = open(save_path+"PStart_"+str(test_frame)+'.dat', 'w')
     pstd.write("PB_TCV_451045\n")
     pstd.write(str(npart)+'\n')
     
     Tst = open(save_path+'TStart_'+str(test_frame)+'.gro', 'w')
     Tst.write("PB_TCV_451045\n")
     Tst.write(str(npart)+"\n")
    
     tstd = open(save_path+"TStart_"+str(test_frame)+'.dat', 'w')
     tstd.write("PB_TCV_451045\n")
     tstd.write(str(npart)+'\n')     

  identities, shiftx, img_nmer, names = mer_identity(inp[0])    
  chemistry += identities
  chain_names += names
  
  # Get a prediction of the trained model 
  pred = model.predict(inp)
  
  # Reverse the normalization 
  pred[0] += 1
  pred[0] /= 2.
  pred[0] *= 0.1711*2
  pred[0] += -0.1711  
  
  c = 0
 
  for ii in range(img_nmer):       
       if identities[ii] == 0: dtseg,merlen,masses = dtseg_cis,merlen_cis,masses_cis
       elif identities[ii] == 1: dtseg,merlen,masses = dtseg_trans,merlen_trans,masses_trans
       elif identities[ii] == 2: dtseg,merlen,masses = dtseg_vinyl,merlen_vinyl,masses_vinyl
       
       P_bv=np.zeros([dtseg,3],dtype=np.float64)
       T_bv=np.zeros([dtseg,3],dtype=np.float64)

       iosx=float(inp[0][c+shiftx+1,0])
       iosy=float(inp[0][c+shiftx+1,1])
       iosz=float(inp[0][c+shiftx+1,2])
       
       j=0 
       i = ua_idx(shiftx,c,j)
       P_bv[j]=[float(pred[0][i,0]),float(pred[0][i,1]),float(pred[0][i,2])] 
       T_bv[j]=[float(tar[i,0]),float(tar[i,1]),float(tar[i,2])] 

       j=1 
       i +=1
       P_bv[j]=[float(pred[0][i,0]),float(pred[0][i,1]),float(pred[0][i,2])] 
       T_bv[j]=[float(tar[i,0]),float(tar[i,1]),float(tar[i,2])] 
       
       j=2 
       i +=1
       P_bv[j]=[float(pred[0][i,0]),float(pred[0][i,1]),float(pred[0][i,2])] 
       T_bv[j]=[float(tar[i,0]),float(tar[i,1]),float(tar[i,2])] 
        
       j=3 
       i +=1
       P_bv[j]=[float(pred[0][i,0]),float(pred[0][i,1]),float(pred[0][i,2])] 
       T_bv[j]=[float(tar[i,0]),float(tar[i,1]),float(tar[i,2])] 
        
       c += dtseg  
       if(identities[ii]==0): 
              cis_d(P_bv,masses,iosx,iosy,iosz,pb0,nmer_count,Pcoords,chemistry,PartIndx)
              cis_d(T_bv,masses,iosx,iosy,iosz,tb0,nmer_count,Tcoords,chemistry,PartIndx)
              PartIndx += merlen 
       elif(identities[ii]==1): 
              trans_d(P_bv,masses,iosx,iosy,iosz,pb0,nmer_count,Pcoords,chemistry,PartIndx)
              trans_d(T_bv,masses,iosx,iosy,iosz,tb0,nmer_count,Tcoords,chemistry,PartIndx) 
              PartIndx += merlen 
       elif(identities[ii]==2): 
              vinyl_d(P_bv,masses,iosx,iosy,iosz,pb0,nmer_count,Pcoords,chemistry,PartIndx)
              vinyl_d(T_bv,masses,iosx,iosy,iosz,tb0,nmer_count,Tcoords,chemistry,PartIndx)
              PartIndx += merlen 
                 
       nmer_count += 1
  if len(chemistry) == nmer :
    for ii in range(Pcoords.shape[0]-1):
        while(Pcoords[ii+1][0]-Pcoords[ii][0]<-hLXX): Pcoords[ii+1][0]+=LXX
        while(Pcoords[ii+1][0]-Pcoords[ii][0]>hLXX): Pcoords[ii+1][0]-=LXX
        while(Pcoords[ii+1][1]-Pcoords[ii][1]<-hLYY): Pcoords[ii+1][1]+=LYY
        while(Pcoords[ii+1][1]-Pcoords[ii][1]>hLYY): Pcoords[ii+1][1]-=LYY
        while(Pcoords[ii+1][2]-Pcoords[ii][2]<-hLZZ): Pcoords[ii+1][2]+=LZZ
        while(Pcoords[ii+1][2]-Pcoords[ii][2]>hLZZ): Pcoords[ii+1][2]-=LZZ       
    for j in range(Pcoords.shape[0]):  
       Pst.write('%5d%5s%5s%5s%8.3f%8.3f%8.3f  0.0000  0.0000  0.0000\n'%((pdimg+1,'PB',chain_names[j],str(j+1+pdimg*chainlen)[-5:],Pcoords[j][0],Pcoords[j][1],Pcoords[j][2]))) 
       print(chain_names[j],Pcoords[j][0],Pcoords[j][1],Pcoords[j][2], file=pstd)   
       Tst.write('%5d%5s%5s%5s%8.3f%8.3f%8.3f  0.0000  0.0000  0.0000\n'%((pdimg+1,'PB',chain_names[j],str(j+1+pdimg*chainlen)[-5:],Tcoords[j][0],Tcoords[j][1],Tcoords[j][2]))) 
       print(chain_names[j],Tcoords[j][0],Tcoords[j][1],Tcoords[j][2], file=tstd)  
    pdimg+=1
    chemistry = []
    chain_names = []
    pb0=np.zeros([nmer,3],dtype=np.float64)
    tb0=np.zeros([nmer,3],dtype=np.float64)
    nmer_count = 0
    PartIndx = 0
   
    if (pdimg%nchain) == 0:  
        nframes+=1
        pdimg=0
        print(nframes)
        Pst.write('%10.5f%10.5f%10.5f\n'%((LXX , LYY , LZZ)))
        Pst.close
        pstd.close
        Tst.write('%10.5f%10.5f%10.5f\n'%((LXX , LYY , LZZ)))
        Tst.close
        tstd.close



### Prediction of an atomistic configuration from a given CG configuration <span id='p'/>

In [57]:
checkpoint_path = "./tmp_c18/cp-0700.ckpt"
model.load_weights(checkpoint_path)
create_chains(test_frame)

1


# Visualize the results <span id='q'/>

### Generate the distribution plots for bond lengths, bond angles and dihedral angles <span id='r'/>

In [60]:
generate_distribution_plots(test_frame)

1
Bond lenghts combinations:  18
Bond angles combinations:  27
Dihedral angles combinations:  42
Bond lenghts combinations:  18
Bond angles combinations:  27
Dihedral angles combinations:  42
4.125962018966675
